In [4]:
import sys
import requests
import pprint
from time import sleep

In [5]:
# store global constants
headers = {
   "authorization": "b7d40c7c26294f6eb9bffc672a93384e",
   "content-type": "application/json"
}
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"
upload_endpoint = 'https://api.assemblyai.com/v2/upload'

In [8]:
# make a function to pass the mp3 to the upload endpoint
def read_file(filename):
   with open(filename, 'rb') as _file:
       while True:
           data = _file.read(5242880)
           if not data:
               break
           yield data

print(sys.argv)

['/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/ashley/Library/Jupyter/runtime/kernel-3014ec5f-2327-434c-b3f9-29360ab80ab6.json']


In [11]:
# upload our audio file
upload_response = requests.post(
   upload_endpoint,
   headers=headers, data=read_file("/Users/ashley/Documents/VSCode/metrohacks2021/arctic_tern_test.mp3")
)
print('Audio file uploaded')

Audio file uploaded


In [12]:
# send a request to transcribe the audio file
transcript_request = {'audio_url': upload_response.json()['upload_url']}
transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers)
print('Transcription Requested')
pprint.pprint(transcript_response.json())
# set up polling
polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
filename = transcript_response.json()['id'] + '.txt'
# if our status isn’t complete, sleep and then poll again
while polling_response.json()['status'] != 'completed':
   sleep(30)
   polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
   print("File is", polling_response.json()['status'])
with open(filename, 'w') as f:
   f.write(polling_response.json()['text'])
print('Transcript saved to', filename)

Transcription Requested
{'acoustic_model': 'assemblyai_default',
 'audio_duration': None,
 'audio_end_at': None,
 'audio_start_from': None,
 'audio_url': 'https://cdn.assemblyai.com/upload/abae7dca-592d-482c-9ed2-559ecfa9dd29',
 'auto_chapters': False,
 'auto_highlights': False,
 'auto_highlights_result': None,
 'boost_param': None,
 'chapters': None,
 'confidence': None,
 'content_safety': False,
 'content_safety_labels': {},
 'disfluencies': False,
 'dual_channel': None,
 'filter_profanity': False,
 'format_text': True,
 'iab_categories': False,
 'iab_categories_result': {},
 'id': '5ci45w5qb-7c60-482d-9b0c-f39ebe00ab1d',
 'language_code': 'en_us',
 'language_model': 'assemblyai_default',
 'punctuate': True,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'sentiment_analysis': False,
 'sentiment_analysis_results': None,
 'speaker_labels': False,
 'speed_boost': False,
 'status': 'queued',
 't